In [53]:
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import numpy as np
import os
from urllib.parse import quote
from selenium import webdriver

In [54]:
# Setting up the Chrome driver
chrome_driver_path = "E:\Major Project 2\IPL Player Performance Prediction\chromedriver-win64\chromedriver.exe"
webdriver_path = "E:\Major Project 2\IPL Player Performance Prediction\chromedriver-win64\Chromedriver"
os.chdir(webdriver_path)

# %cd webdriver_path

driver = webdriver.Chrome(chrome_driver_path)

In [55]:
# Extracting recent stats of the players
final_data = pd.DataFrame()  # Final dataframe to store
# players= ["RG Sharma","SA Yadav", "TH David", "Ishan Kishan", "JJ Bumrah", "D Madushanka", "HH Pandya", "NT Tilak Varma","PP Chawla", "G Coetzee", "Mohammad Nabi"]  # all the player data [MI]
# players = ['DA Warner', 'PP Shaw',  'HC Brook', 'RR Pant', 'A Nortje', 'Kuldeep Yadav', 'Mukesh Kumar', 'KK Ahmed', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh'] # [DC]
# players = ['D Padikkal', 'A Badoni', 'KL Rahul', 'Q de Kock', 'N Pooran', 'MA Wood', 'Mohsin Khan', 'Ravi Bishnoi', 'Yash Thakur', 'Shivam Mavi', 'DJ Hooda', 'KH Pandya', 'MP Stoinis'] # [LSG]
# players = ['B Sai Sudharsan', 'DA Miller', 'Shubman Gill', 'WP Saha', 'Mohammed Shami', 'MM Sharma', 'Noor Ahmad', 'SH Johnson', 'R Tewatia', 'Rashid Khan', 'V Shankar', 'M Shahrukh Khan'] #[GT]
# players = ['YBK Jaiswal', 'R Parag', 'SO Hetmyer', 'R Powell', 'SV Samson', 'DC Jurel', 'JC Buttler', 'YS Chahal', 'M Prasidh Krishna', 'TA Boult', 'R Ashwin'] #[RR]
# players = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]
# players = ['N Rana', 'RK Singh', 'SS Iyer', 'Rahmanullah Gurbaz', 'VG Arora', 'CV Varun', 'C Sakariya', 'MA Starc', 'AD Russell', 'SP Narine', 'VR Iyer'] #[KKR]
# players = ['F du Plessis', 'V Kohli', 'RM Patidar', 'KD Karthik', 'MJ Dagar', 'Mohammed Siraj', 'V Vyshak', 'AS Joseph', 'Yash Dayal', 'GJ Maxwell', 'Cameron Green'] #[RCB]
# players = ["AK Markram", "RA Tripathi", "MA Agarwal", "TM Head", "H Klaasen", "B Kumar", "Umran Malik", "PJ Cummins", "Shahbaz Ahmed", "Washington Sundar", "T Natarajan"] #[SRH]
players = ['AM Rahane', 'RD Gaikwad', 'DP Conway', 'MS Dhoni', 'DL Chahar', 'M Theekshana', 'M Pathirana', 'SN Thakur', 'RA Jadeja', 'S Dube', 'DJ Mitchell'] #[CSK]

In [56]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Looping through all the players
for i in players[0:]:
   # Encode the player name for URL
    encoded_player_name = quote(i)
    
    # Accessing the web page for the current player's stats
    url = f"http://www.cricmetric.com/playerstats.py?" \
          f"player={encoded_player_name}&role=all&format=TWENTY20&" \
          f"groupby=match&playerStatsFilters=on&start_date=2015-01-01&" \
          f"end_date=2023-05-18&start_over=0&end_over=9999"
    
    driver.get(url)
    
    # Scrolling down to load all the stats
    # driver.execute_script("window.scrollTo(0, 500)")
    driver.maximize_window()
    time.sleep(3)
    
    try:
        # Extracting batting stats of the player
        # Wait for the batting table to be present
        batting_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Batting"]/div/table'))
        )
        
        # batting_table = driver.find_element_by_xpath(
        #     '//*[@id="TWENTY20-Batting"]/div/table')
        
        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%'] 
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']
        
        # driver.execute_script("arguments[0].scrollIntoView();", bowling_tab)

        # Switching to bowling stats tab
        bowling_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Bowling-tab"]'))
        )
        # bowling_tab = WebDriverWait(driver, 10).until(
        #     EC.element_to_be_clickable((By.XPATH, '//*[@id="TWENTY20-Bowling-tab"]'))
        # )

        bowling_tab.click()
        time.sleep(5)

        # Extracting bowling stats of the player
        bowling_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Bowling"]/div/table'))
        )
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']
        
    except:
        # If stats for current player not found,
        # create empty dataframe
        # print("I m in except")
        stats2 = pd.DataFrame({'Match':pd.Series(stats['Match'][0:1]),
                               'Overs Bowled':[0],'Runs Given':[0],
                               'Wickets Taken':[0],'Econ':[0],
                               'Bowl Avg':[0],'Bowl SR':[0],'5W':[0],
                               '4s Given':[0],'6s Given':[0],
                               'Bowl Dot%':[0]})
        
    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    # print(merged_stats)
    merged_stats = merged_stats.sort_values(by=['Match'])
    
    # Create lagged variables for future performance prediction
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_data = final_data.append(merged_stats)
    
final_data = final_data[final_data['Match']!=0]

final_data['Bowl Avg'] = np.where(final_data['Bowl Avg']=='-',
                                 0,final_data['Bowl Avg'])
final_data['Bowl SR'] = np.where(final_data['Bowl SR']=='-',
                                0,final_data['Bowl SR'])
final_data = final_data[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_data = final_data.replace('-',0)
final_data = final_data.fillna(0)
# print(final_data)
final_data
final_data.to_csv('E:/Major Project 2/IPL Player Performance Prediction/Players data\output_file.csv', index=False, mode='w')


In [59]:
import pandas as pd
import numpy as np
import math
import time
import math
from statistics import stdev
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

import scipy.stats

models = pd.DataFrame()
# players_list = ["RG Sharma","SA Yadav", "TH David", "Ishan Kishan", "JJ Bumrah", "D Madushanka", "HH Pandya", "NT Tilak Varma","PP Chawla", "G Coetzee", "Mohammad Nabi"] #[MI]
# players_list = ['DA Warner', 'PP Shaw',  'HC Brook', 'RR Pant', 'A Nortje', 'Kuldeep Yadav', 'Mukesh Kumar', 'KK Ahmed', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh'] #[DC]
# players_list = ['D Padikkal', 'A Badoni', 'KL Rahul', 'Q de Kock', 'N Pooran', 'MA Wood', 'Mohsin Khan', 'Ravi Bishnoi', 'Yash Thakur', 'Shivam Mavi', 'DJ Hooda', 'KH Pandya', 'MP Stoinis'] #[LSG]
# players_list = ['B Sai Sudharsan', 'DA Miller', 'Shubman Gill', 'WP Saha', 'Mohammed Shami', 'MM Sharma', 'Noor Ahmad', 'SH Johnson', 'R Tewatia', 'Rashid Khan', 'V Shankar', 'M Shahrukh Khan'] #[GT]
# players_list = ['YBK Jaiswal', 'R Parag', 'SO Hetmyer', 'R Powell', 'SV Samson', 'DC Jurel', 'JC Buttler', 'YS Chahal', 'M Prasidh Krishna', 'TA Boult', 'R Ashwin'] #[RR]
# players_list = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]
# players_list = ['N Rana', 'RK Singh', 'SS Iyer', 'Rahmanullah Gurbaz', 'VG Arora', 'CV Varun', 'C Sakariya', 'MA Starc', 'AD Russell', 'SP Narine', 'VR Iyer'] #[KKR]
# players_list = ['F du Plessis', 'V Kohli', 'RM Patidar', 'KD Karthik', 'MJ Dagar', 'Mohammed Siraj', 'V Vyshak', 'AS Joseph', 'Yash Dayal', 'GJ Maxwell', 'Cameron Green'] #[RCB]
# players_list = ["AK Markram", "RA Tripathi", "MA Agarwal", "TM Head", "H Klaasen", "B Kumar", "Umran Malik", "PJ Cummins", "Shahbaz Ahmed", "Washington Sundar", "T Natarajan"] #[SRH]
players_list = ['AM Rahane', 'RD Gaikwad', 'DP Conway', 'MS Dhoni', 'DL Chahar', 'M Theekshana', 'M Pathirana', 'SN Thakur', 'RA Jadeja', 'S Dube', 'DJ Mitchell'] #[CSK]


csv_file = 'E:\Major Project 2\IPL Player Performance Prediction\Players data\output_file.csv'
   
# Read the CSV file into a DataFrame
final_data = pd.read_csv(csv_file)

total_accuracy = 0
num_models = 0
avg_sum = 0

for player_name in players_list:
    # Filter the data for the current player
    player_data = final_data[final_data['Player'] == player_name]

    # Remove rows with missing values
    player_new = player_data.dropna()

    # Predict next runs
    X_runs = player_new[player_new.columns[2:11]]
    y_runs = player_new[player_new.columns[21:22]]

    X_train_runs, X_test_runs, y_train_runs,y_test_runs = train_test_split(X_runs, y_runs, test_size=0.4, random_state=123)
    ridge_runs = pd.DataFrame()

    # Use a pipeline to scale the data and train the model
    pipeline = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=123))
    pipeline.fit(X_train_runs, y_train_runs.values.ravel())

    # Calculate accuracy
    train_accuracy_runs = pipeline.score(X_train_runs, y_train_runs)
    test_accuracy_runs = pipeline.score(X_test_runs, y_test_runs)

    print("Runs Prediction Accuracy:")
    print("Training Accuracy:", train_accuracy_runs)
    print("Testing Accuracy:", test_accuracy_runs)

    # --------------------------------------------
    # Predict the 'next_runs' for the testing set
    y_pred_best = pipeline.predict(X_test)

    # Calculate the mean squared error of the predictions
    mse_best = mean_squared_error(y_test, y_pred_best)
    print("Mean Squared Error with Best Parameters: ", mse_best) 

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred_best)
    print("Mean Absolute Error with Best Parameters: ", mae)

    # Calculate R-squared
    r2 = r2_score(y_test, y_pred_best)
    print("R-squared with Best Parameters: ", r2)
    # --------------------------------------------

    #  # For runs prediction
    # y_pred_runs = next_runs.predict(X_test_runs)
    # mse_runs = mean_squared_error(y_test_runs, y_pred_runs)
    # rmse_runs = mean_squared_error(y_test_runs, y_pred_runs, squared=False)
    # mae_runs = mean_absolute_error(y_test_runs, y_pred_runs)
    # r2_runs = r2_score(y_test_runs, y_pred_runs)

    # print("Runs Prediction Metrics:")
    # print("MSE:", mse_runs)
    # print("RMSE:", rmse_runs)
    # print("MAE:", mae_runs)
    # print("R2:", r2_runs)
# --------------------------------------------------------
# # Calculate the average accuracy
# # average_accuracy = total_accuracy / num_models
# # print("Overall Accuracy:", average_accuracy)
# print(avg_sum/11)

Runs Prediction Accuracy:
Training Accuracy: 0.8382043537573081
Testing Accuracy: -0.45731181395414766
Mean Squared Error with Best Parameters:  525.2339279215686
Mean Absolute Error with Best Parameters:  20.61886274509804
R-squared with Best Parameters:  -1.088620798741446
Runs Prediction Accuracy:
Training Accuracy: 0.8091022887967665
Testing Accuracy: -0.1861769108521265
Mean Squared Error with Best Parameters:  646.8149176470588
Mean Absolute Error with Best Parameters:  21.64588235294118
R-squared with Best Parameters:  -1.5720941053442679
Runs Prediction Accuracy:
Training Accuracy: 0.8315595509225555
Testing Accuracy: -0.9926165423047577
Mean Squared Error with Best Parameters:  474.7233
Mean Absolute Error with Best Parameters:  18.37235294117647
R-squared with Best Parameters:  -0.8877625859981286
Runs Prediction Accuracy:
Training Accuracy: 0.8221978903885592
Testing Accuracy: -0.2391863052391785
Mean Squared Error with Best Parameters:  431.35745306045754
Mean Absolute Erro

In [74]:
import pandas as pd
import numpy as np
import math
import time
import math
from statistics import stdev
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

import scipy.stats

models = pd.DataFrame()
# players_list = ["RG Sharma","SA Yadav", "TH David", "Ishan Kishan", "JJ Bumrah", "D Madushanka", "HH Pandya", "NT Tilak Varma","PP Chawla", "G Coetzee", "Mohammad Nabi"] #[MI]
# players_list = ['DA Warner', 'PP Shaw',  'HC Brook', 'RR Pant', 'A Nortje', 'Kuldeep Yadav', 'Mukesh Kumar', 'KK Ahmed', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh'] #[DC]
# players_list = ['D Padikkal', 'A Badoni', 'KL Rahul', 'Q de Kock', 'N Pooran', 'MA Wood', 'Mohsin Khan', 'Ravi Bishnoi', 'Yash Thakur', 'Shivam Mavi', 'DJ Hooda', 'KH Pandya', 'MP Stoinis'] #[LSG]
# players_list = ['B Sai Sudharsan', 'DA Miller', 'Shubman Gill', 'WP Saha', 'Mohammed Shami', 'MM Sharma', 'Noor Ahmad', 'SH Johnson', 'R Tewatia', 'Rashid Khan', 'V Shankar', 'M Shahrukh Khan'] #[GT]
# players_list = ['YBK Jaiswal', 'R Parag', 'SO Hetmyer', 'R Powell', 'SV Samson', 'DC Jurel', 'JC Buttler', 'YS Chahal', 'M Prasidh Krishna', 'TA Boult', 'R Ashwin'] #[RR]
# players_list = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]
# players_list = ['N Rana', 'RK Singh', 'SS Iyer', 'Rahmanullah Gurbaz', 'VG Arora', 'CV Varun', 'C Sakariya', 'MA Starc', 'AD Russell', 'SP Narine', 'VR Iyer'] #[KKR]
# players_list = ['F du Plessis', 'V Kohli', 'RM Patidar', 'KD Karthik', 'MJ Dagar', 'Mohammed Siraj', 'V Vyshak', 'AS Joseph', 'Yash Dayal', 'GJ Maxwell', 'Cameron Green'] #[RCB]
# players_list = ["AK Markram", "RA Tripathi", "MA Agarwal", "TM Head", "H Klaasen", "B Kumar", "Umran Malik", "PJ Cummins", "Shahbaz Ahmed", "Washington Sundar", "T Natarajan"] #[SRH]
players_list = ['AM Rahane', 'RD Gaikwad', 'DP Conway', 'MS Dhoni', 'DL Chahar', 'M Theekshana', 'M Pathirana', 'SN Thakur', 'RA Jadeja', 'S Dube', 'DJ Mitchell'] #[CSK]


csv_file = 'E:\Major Project 2\IPL Player Performance Prediction\Players data\output_file.csv'
   
# Read the CSV file into a DataFrame
final_data = pd.read_csv(csv_file)

total_accuracy = 0
num_models = 0
avg_sum = 0

for player_name in players_list:
    # Filter the data for the current player
    player_data = final_data[final_data['Player'] == player_name]

    # Remove rows with missing values
    player_new = player_data.dropna()

    # Predict next runs
    X_runs = player_new[player_new.columns[2:11]]
    y_runs = player_new[player_new.columns[21:22]]

    X_train, X_test, y_train,y_test = train_test_split(X_runs, y_runs, test_size=0.5, random_state=123)
    ridge_runs = pd.DataFrame()


    # # Function to create model, required for KerasClassifier
    # def create_model(optimizer='adam'):
    #     model = Sequential()
    #     model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    #     model.add(Dense(16, activation='relu'))
    #     model.add(Dense(1))
    #     model.compile(loss='mean_squared_error', optimizer=optimizer)
    #     return model

    # # Create the KerasRegressor model
    # model = KerasRegressor(build_fn=create_model, verbose=0)

    # # Define the grid search parameters
    # param_grid = {
    #     'batch_size': [10, 20, 30, 40],
    #     'epochs': [10, 50, 100],
    #     'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    # }

    # # Create Grid Search
    # grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    # grid_result = grid.fit(X_train, y_train)

    # # Summarize results
    # print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# ------------------
    train_mses = []
    test_mses = []  
    mae = []  
    r_squares = []  

    # Define the model
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))  # Input layer
    model.add(Dense(16, activation='relu'))  # Hidden layer
    model.add(Dense(1))  # Output layer

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')

    # Fit the model
    model.fit(X_train, y_train, epochs=20, batch_size=30, verbose=1)

    # Evaluate the model
    mse_train = model.evaluate(X_train, y_train)
    mse_test = model.evaluate(X_test, y_test)

    print("Train MSE: ", mse_train)
    print("Test MSE: ", mse_test)

    # Predict 'next_runs'
    y_pred_nn = model.predict(X_test)

    # Calculate MAE and R-squared
    mae_nn = mean_absolute_error(y_test, y_pred_nn)
    r2_nn = r2_score(y_test, y_pred_nn)

    print("Mean Absolute Error with Neural Network: ", mae_nn)
    print("R-squared with Neural Network: ", r2_nn)
    train_mses.append(mse_train)
    test_mses.append(mse_test)
    mae.append(mae_nn)
    r_squares.append(r2_nn)

# Calculate the averages
avg_train_mse = sum(train_mses) / len(train_mses)
avg_test_mse = sum(test_mses) / len(test_mses)
avg_r_square = sum(r_squares) / len(r_squares)
avg_r_square = sum(mae) / len(mae)

print("Average Train MSE: ", avg_train_mse)
print("Average Test MSE: ", avg_test_mse)
print("Average R-squared: ", avg_r_square)


Epoch 1/20


2/2 [==============================] - 1s 6ms/step - loss: 858.1195
Epoch 2/20
2/2 [==============================] - 0s 6ms/step - loss: 754.5115
Epoch 3/20
2/2 [==============================] - 0s 7ms/step - loss: 720.4235
Epoch 4/20
2/2 [==============================] - 0s 5ms/step - loss: 697.3872
Epoch 5/20
2/2 [==============================] - 0s 5ms/step - loss: 677.4267
Epoch 6/20
2/2 [==============================] - 0s 5ms/step - loss: 662.3383
Epoch 7/20
2/2 [==============================] - 0s 6ms/step - loss: 644.1611
Epoch 8/20
2/2 [==============================] - 0s 5ms/step - loss: 633.0209
Epoch 9/20
2/2 [==============================] - 0s 7ms/step - loss: 640.9813
Epoch 10/20
2/2 [==============================] - 0s 6ms/step - loss: 611.8303
Epoch 11/20
2/2 [==============================] - 0s 5ms/step - loss: 597.9084
Epoch 12/20
2/2 [==============================] - 0s 6ms/step - loss: 595.8582
Epoch 13/20
2/2 [==============================] - 0s 7ms/st

In [58]:
# Adjusting values based on conditions and rounding

# Adjusting next_runs_given based on next_overs
models['next_runs_given'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given'] / models['next_overs'] * 4,
    models['next_runs_given']
)
models['next_runs_given_ll_95'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given_ll_95'] / models['next_overs'] * 4,
    models['next_runs_given_ll_95']
)
models['next_runs_given_ul_95'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given_ul_95'] / models['next_overs'] * 4,
    models['next_runs_given_ul_95']
)

# Limiting next_overs to a maximum of 4
models['next_overs'] = np.where(
    models['next_overs'] > 4,
    4,
    models['next_overs']
)
models['next_overs_ll_95'] = np.where(
    models['next_overs_ll_95'] > 4,
    4,
    models['next_overs_ll_95']
)
models['next_overs_ul_95'] = np.where(
    models['next_overs_ul_95'] > 4,
    4,
    models['next_overs_ul_95']
)

# Adjusting next_runs based on next_balls
models['next_runs'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs_ul_95']
)

# Setting next_runs to a minimum of 1
models['next_runs'] = np.where(
    models['next_runs'] < 0,
    1,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_runs_ll_95'] < 0,
    1,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_runs_ul_95'] < 0,
    1,
    models['next_runs_ul_95']
)

# Adjusting next_runs based on next_balls if next_balls > 100
models['next_runs'] = np.where(
    models['next_balls'] > 100,
    models['next_runs'] / models['next_balls'] * 5,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_balls'] > 100,
    models['next_runs_ll_95'] / models['next_balls'] * 5,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_balls'] > 100,
    models['next_runs_ul_95'] / models['next_balls'] * 5,
    models['next_runs_ul_95']
)

# Limiting next_balls to a maximum of 5
models['next_balls'] = np.where(
    models['next_balls'] > 100,
    5,
    models['next_balls']
)
models['next_balls_ll_95'] = np.where(
    models['next_balls_ll_95'] > 100,
    5,
    models['next_balls_ll_95']
)
models['next_balls_ul_95'] = np.where(
    models['next_balls_ul_95'] > 100,
    5,
    models['next_balls_ul_95']
)

# Setting next_balls to a minimum of 1
models['next_balls'] = np.where(
    models['next_balls'] < 0,
    1,
    models['next_balls']
)
models['next_balls_ll_95'] = np.where(
    models['next_balls_ll_95'] < 0,
    1,
    models['next_balls_ll_95']
)
models['next_balls_ul_95'] = np.where(
    models['next_balls_ul_95'] < 0,
    1,
    models['next_balls_ul_95']
)

# Setting next_wkts to a minimum of 1
models['next_wkts'] = np.where(
    models['next_wkts'] < 0,
    1,
    models['next_wkts']
)
models['next_wkts_ll_95'] = np.where(
    models['next_wkts_ll_95'] < 0,
    1,
    models['next_wkts_ll_95']
)
models['next_wkts_ul_95'] = np.where(
    models['next_wkts_ul_95'] < 0,
    1,
    models['next_wkts_ul_95']
)

# Rounding values to 0 decimal places
models['next_runs'] = round(models['next_runs'], 0)
models['next_runs_ll_95'] = round(models['next_runs_ll_95'], 0)
models['next_runs_ul_95'] = round(models['next_runs_ul_95'], 0)

models['next_balls'] = round(models['next_balls'], 0)
models['next_balls_ll_95'] = round(models['next_balls_ll_95'], 0)
models['next_balls_ul_95'] = round(models['next_balls_ul_95'], 0)

models['next_wkts'] = round(models['next_wkts'], 0)
models['next_wkts_ll_95'] = round(models['next_wkts_ll_95'], 0)
models['next_wkts_ul_95'] = round(models['next_wkts_ul_95'], 0)

models['next_runs_given'] = round(models['next_runs_given'], 0)
models['next_runs_given_ll_95'] = round(models['next_runs_given_ll_95'], 0)
models['next_runs_given_ul_95'] = round(models['next_runs_given_ul_95'], 0)

models['next_overs'] = round(models['next_overs'], 0)
models['next_overs_ll_95'] = round(models['next_overs_ll_95'], 0)
models['next_overs_ul_95'] = round(models['next_overs_ul_95'], 0)

# print(models)
models.to_csv('E:/Major Project 2/IPL Player Performance Prediction/models2_SRH.csv', index=False)

KeyError: 'next_overs'